In [1]:
import argparse
import json

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from fastchat.conversation import get_default_conv_template, compute_skip_echo_len
from fastchat.serve.inference import load_model


/data/jason.tangth/py39_vicuna/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model_path = '/data/jason.tangth/model/vicuna_data/vicuna-13b/'

In [7]:
model, tokenizer = load_model(
    '/data/jason.tangth/model/vicuna_data/vicuna-13b/',
    'cuda',
    '1',
    '28Gib',
    False,
    False,
    )

init_kwargs {'torch_dtype': torch.float16}


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:46<00:00, 35.59s/it]


In [38]:
msg = '''
Choose a category for the product "dek lumpur dan tutup mesin beat karbu 2008 - 2012 terbaru!!" from below options:

Motorcycles - Motorcycle Accessories - Mud Flaps & Splash Guards
Motorcycles - Motorcycle Spare Parts - Body & Frame
Motorcycles - Motorcycle Spare Parts - Others
Motorcycles - Motorcycle Accessories - Others
Motorcycles - Motorcycles
Motorcycles - Others
Motorcycles - Motorcycle Accessories - Boxes & Cases
Motorcycles - Motorcycle Accessories - Carpets
Motorcycles - Motorcycle Spare Parts - Cables & Tubes
Automobiles - Automobile Exterior Accessories - Mud Flaps & Splash Guards
Motorcycles - Motorcycle Accessories - Covers
Motorcycles - Motorcycle Spare Parts - Tires & Accessories
Motorcycles - Motorcycle Spare Parts - Drivetrain, Transmission & Clutches - Chains & Gears
Motorcycles - Motorcycle Accessories - Stickers, Logos & Emblems
Motorcycles - Motorcycle Spare Parts - Batteries & Accessories
None of above

Choose a correct answer from above options in English. Do not provide answer not shown above.
'''
conv = get_default_conv_template(model_path).copy()
conv.append_message(conv.roles[0], msg)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

inputs = tokenizer([prompt])
output_ids = model.generate(
    torch.as_tensor(inputs.input_ids).cuda(),
    do_sample=True,
    temperature=0.05,
    max_new_tokens=512,
)
outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
skip_echo_len = compute_skip_echo_len(model_path, conv, prompt)
outputs = outputs[skip_echo_len:]

# print(f"{conv.roles[0]}: {msg}")
print(f"{outputs}")

In [39]:
q = 
'''
What category of the product "dek lumpur dan tutup mesin beat karbu 2008 - 2012 terbaru!!" belongs to?

Motorcycles - Motorcycle Accessories - Mud Flaps & Splash Guards
Motorcycles - Motorcycle Spare Parts - Body & Frame
Motorcycles - Motorcycle Spare Parts - Others
Motorcycles - Motorcycle Accessories - Others
Motorcycles - Motorcycles
Motorcycles - Others
Motorcycles - Motorcycle Accessories - Boxes & Cases
Motorcycles - Motorcycle Accessories - Carpets
Motorcycles - Motorcycle Spare Parts - Cables & Tubes
Automobiles - Automobile Exterior Accessories - Mud Flaps & Splash Guards
Motorcycles - Motorcycle Accessories - Covers
Motorcycles - Motorcycle Spare Parts - Tires & Accessories
Motorcycles - Motorcycle Spare Parts - Drivetrain, Transmission & Clutches - Chains & Gears
Motorcycles - Motorcycle Accessories - Stickers, Logos & Emblems
Motorcycles - Motorcycle Spare Parts - Batteries & Accessories
None of above

Choose a correct answer from above options in English. Do not provide answer not shown above.
'''

Motorcycles - Motorcycle Accessories - Others


In [ ]:
! curl http://localhost:8945/v1/chat/completions -H "Content-Type: application/json" -d '{"model": "vicuna-13b-v1.1","messages": [{"role": "user", "content": "Hello!"}]}'

In [43]:
# test on monthly QC
import pandas as pd
import pickle 
df = pd.read_csv('./CSPU_Cluster_Monthly_wrong_category_12_sorted_gpt4_gpt35_gpt35top15_gpt35top30_gpt4top15_gpt35top1510in1.csv')

In [55]:
with open('/data/jason.tangth/codebase/offline-and-online-data-filter-and-category-prediction-v1.9.7/assets/dictionary/data_dict.pkl', 
          'rb') as f:
    data_dict = pickle.load(f)

leaf_to_L1_dict = data_dict['leaf_to_L1_dict']
ID_mapping_dict = data_dict['ID_mapping_dict']
name_mapping_dict = data_dict['name_mapping_dict']

In [134]:
## prompt 1
def get_vicuna_result(title, categories, model_path='/data/jason.tangth/model/vicuna_data/vicuna-13b/'):
    msg = '''
        Choose a category for the product "{product_title}" from below options:
        
        {categories}        
        
        Just output the option.
        '''.format(product_title=title, categories=categories)
    conv = get_default_conv_template(model_path).copy()
    conv.append_message(conv.roles[0], msg)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    inputs = tokenizer([prompt])
    output_ids = model.generate(
        torch.as_tensor(inputs.input_ids).cuda(),
        do_sample=True,
        temperature=0.05,
        max_new_tokens=128,
    )
    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
    skip_echo_len = compute_skip_echo_len(model_path, conv, prompt)
    outputs = outputs[skip_echo_len:]
    return outputs

def get_categories(candidates):
    target_leaf_name = [ID_mapping_dict.get(c).replace('BE - ','') for c in candidates]
    categories = ''
    for i in range(len(target_leaf_name)):
        categories += '{}\n'.format(target_leaf_name[i])
    categories += 'None of above\n'
    return categories

gpt_results = []
for i, row in df.iterrows():
    candidates = eval(row['candidates'].replace(' ',','))
    categories = get_categories(candidates)
    title = row['title']
    r = get_vicuna_result(title, categories)
    gpt_results.append(r)

In [135]:
gpt_results

['Beauty - Makeup - Face - Concealer & Corrector',
 'Beauty - Makeup - Face - Concealer & Corrector',
 'Beauty - Makeup - Face - Concealer & Corrector',
 'Beauty - Makeup - Face - Concealer & Corrector',
 'Beauty - Makeup - Face - Concealer & Corrector',
 'Beauty - Makeup - Face - Concealer & Corrector',
 'Beauty - Makeup - Face - Concealer & Corrector',
 'Beauty - Makeup - Face - Concealer & Corrector',
 'Beauty - Makeup - Face - Concealer & Corrector',
 'Beauty - Makeup - Face - Concealer & Corrector',
 'Women Bags - Wallets - Coin Holders & Purses',
 'Beauty - Beauty Tools - Makeup Accessories - Makeup Bags & Organizers',
 'Women Bags - Wallets - Coin Holders & Purses',
 'Men Bags - Wallets - Coin Holders & Purses',
 'Women Bags - Wallets - Coin Holders & Purses',
 'Women Bags - Wallets - Coin Holders & Purses',
 'Women Bags - Wallets - Coin Holders & Purses',
 'Women Bags - Wallets - Coin Holders & Purses',
 'Output the option text:\n\nMen Bags - Wallets - Coin Holders & Purses\nWo

In [136]:
df['vicuna_top15_results'] = gpt_results

In [137]:
import numpy as np
leaf_node = np.array([i.replace('BE - ', '') for i in name_mapping_dict.keys() if 'BE - ' in i])

In [139]:
rest = df[df['gpt_id']==999999]['vicuna_top15_results'].apply(lambda x:leaf_node[[leaf in x for leaf in leaf_node]])

In [140]:
df.loc[df['gpt_id']==999999, 'vicuna_top15_results'] = rest.apply(lambda x:x[0] if len(x)>0 else '999999').tolist()

In [141]:
df['gpt_id'] = df['vicuna_top15_results'].apply(lambda x:
                                                name_mapping_dict.get(
                                                    'BE - '+x.split('\n')[0],
                                                    999999))

In [142]:
df_local_pic = pd.read_csv('CSPU_Cluster_Monthly_wrong_category_12_sorted_local_gt.csv')
df_local_pic = df_local_pic[df_local_pic['gt_name'].notna()]
df_local_pic['gt_id'] = df_local_pic['gt_id'].astype(int)

In [143]:
gpt_id_col = 'gpt_id'
m_df = df_local_pic.merge(df[['Itemid', gpt_id_col, 'candidates', 'gpt35_top15_results']], on='Itemid')

(m_df[gpt_id_col]==m_df['gt_id']).mean()

0.24237560192616373

In [131]:
df.to_csv('./CSPU_Cluster_Monthly_wrong_category_12_sorted_gpt4_gpt35_gpt35top15_gpt35top30_gpt4top15_gpt35top1510in1_vicuna.csv',
          index=False
         )